In [1]:
import numpy as np
import pandas as pd
import pyxdf

# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from EEGModels import EEGNet
import tensorflow
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

#Sklearn imports
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.utils import compute_class_weight
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from pyriemann.utils.viz import plot_confusion_matrix

In [306]:
#Testing of the individual model

kernels, chans, samples = 1, 16, 1233

In [307]:
model = EEGNet(nb_classes = 1, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 125, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001),
              metrics = ['accuracy'])

In [308]:
X_train = np.loadtxt("X_train_P20_5.csv")
Y_train = np.loadtxt("Y_train_P20_5.csv")

X_test = np.loadtxt("X_test_P20_5.csv")
Y_test = np.loadtxt("Y_test_P20_5.csv")

X_train = X_train.reshape(
     X_train.shape[0], X_train.shape[1] // samples, samples)

X_test = X_test.reshape(
     X_test.shape[0], X_test.shape[1] // samples, samples)

In [310]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
X_test = scaler.fit_transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

randomize_train = np.arange(len(X_train))
np.random.shuffle(randomize_train)
X_train = X_train[randomize_train]
Y_train = Y_train[randomize_train]

randomize_test = np.arange(len(X_test))
np.random.shuffle(randomize_train)
X_test = X_test[randomize_test]
Y_test = Y_test[randomize_test]

In [311]:
X_train  = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_test  = X_test.reshape(X_test.shape[0], chans, samples, kernels)

In [ ]:
fittedModel = model.fit(X_train, Y_train, batch_size = 20, epochs = 20)

In [ ]:
score = model.evaluate(X_test, Y_test)

pred_val = model.predict(X_test)
pred_val = np.where(pred_val > 0.5, 1, 0)

labels = ["Non-Alcohol", "Alcohol"]
cm = confusion_matrix(Y_test, pred_val)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()

tn = cm[0][0] #true negatives
fn = cm[0][1] #false positives

accuracy = score[1]
F1_score = f1_score(y_true=Y_test, y_pred=pred_val)
precision= precision_score(y_true=Y_test, y_pred=pred_val)
recall = recall_score(y_true=Y_test, y_pred=pred_val)
specificity = tn/(tn+fn)

In [ ]:
print("The accuracy of the model with cross validation is", accuracy)
print("The precision score of the model with cross validation is", precision)
print("The recall score of the model with cross validation is", recall)
print("The F1 score of the model with cross validation is", F1_score)
print("The specificity score of the model with cross validation is", specificity)